In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
class RBFKernelActivation(nn.Module):
    def __init__(self, gamma=1.0):
        super(RBFKernelActivation, self).__init__()
        self.gamma = gamma

    def forward(self, x):
        return torch.exp(-self.gamma * torch.pow(x, 2))

In [3]:
# Load and preprocess the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

In [4]:
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
# Convert to PyTorch tensors
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)


In [6]:
# Define the neural network
class KernelNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(KernelNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 10)
        self.kernel_activation = RBFKernelActivation(gamma=0.5)
        self.fc2 = nn.Linear(10, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.kernel_activation(x)  # Apply RBF Kernel Activation
        x = self.fc2(x)
        return x


In [7]:
# Initialize model
model = KernelNN(input_dim=4, output_dim=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [11]:
# Training loop
epochs = 50
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")

Epoch [0/50], Loss: 0.0886
Epoch [10/50], Loss: 0.0832
Epoch [20/50], Loss: 0.0788
Epoch [30/50], Loss: 0.0751
Epoch [40/50], Loss: 0.0720


In [12]:
# Evaluate the model
with torch.no_grad():
    test_outputs = model(X_test)
    test_predictions = torch.argmax(test_outputs, axis=1)
    accuracy = (test_predictions == y_test).float().mean()
    print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 1.00
